In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 
import h5py

import sys
sys.path.append('../python/')
from edw_data_util import *

GGA3/NR/4.0/5.556E-02/0.0380/000/0.1493/0.1782/0.9975/
GGA3/NR/4.0/5.556E-02/0.0381/000/0.1537/0.1703/0.9948/


In [10]:
filename = 'data/edelweiss_corr1_systematicErrors_sampler_nll_allpars_corrAB_gausPrior_2844a28.h5'

In [11]:
f = h5py.File(filename,'r')

# need to store data in an array:
# The sampler will now have a chains attribute 
# which is an array with shape (Nwalker,N,Ndim) 
# where N is the number of interations (500 in our inital run)
# and Ndim is the number of fit parameters
path='{}/{}/'.format('mcmc','sampler')

aH = np.asarray(f[path+'aH'])
C = np.asarray(f[path+'C'])
m = np.asarray(f[path+'m'])
scale = np.asarray(f[path+'scale'])
A = np.asarray(f[path+'A'])
B = np.asarray(f[path+'B'])
samples = np.asarray(f[path+'samples'])

f.close()

In [26]:
print (np.shape(samples))
print (samples[:,:,1])
aH_interval = hpd(np.ndarray.flatten(samples[:,:,1]), 0.6827)
print (aH_interval)

(100, 5000, 6)
[[0.03383839 0.03422393 0.03422393 ... 0.03305807 0.03305807 0.03305807]
 [0.03523041 0.03665635 0.03665635 ... 0.03565165 0.03394058 0.03491631]
 [0.03136095 0.0314244  0.0314244  ... 0.0363116  0.0363116  0.03627199]
 ...
 [0.03449717 0.03260372 0.03260372 ... 0.03084531 0.02891799 0.02989469]
 [0.03540461 0.03540461 0.03559485 ... 0.02874892 0.02874892 0.02783924]
 [0.02946945 0.02946945 0.02946945 ... 0.03005696 0.03008158 0.03008158]]
[0.02995405 0.03598121]


In [ ]:
sampsize=10000
Erecoil_keV = np.linspace(10, 200, 50)
evec = np.zeros((np.size(Erecoil_keV),sampsize))

for i, (aH, C, m, scale, A, B) in enumerate(samples[np.random.randint(len(samples), size=sampsize)]):
    if aH >= aH_interval[0] and aH <= aH_interval[1]:
        V = scale*4.0
        model_NR_0 = [series_NRQ_var_corr1(x, 0, V, aH, 1/18.0, A, B, 'GGA3') for x in Erecoil_keV] 
        model_NR = np.sqrt(np.power(C + m*Erecoil, 2) + model_NR_0)

        v = model_NR
        #print(np.std(v))
        #print(np.mean(v))
        evec[:,i] = v

# use the best-fit function instead of the mean
# to calculate the up and down limits:
# find the height at which 68% of the function is contained
# the upper and lower are the values that cross that height
upvec = []
dnvec = []
for sample in evec:
    #print (np.size(sample))
    #print(sample)
    interval = hpd(sample, 0.6827)
    #print (interval)
    upvec.append(interval[0])
    dnvec.append(interval[1])

## calculate the best-fit values
model_NR_0 = [series_NRQ_var_corr1(x, 0, V_fit, aH_fit, 1/18.0, A_fit, B_fit, 'GGA3') for x in Erecoil_keV] 
model_NR_fit = np.sqrt(np.power(C_fit + m_fit*Erecoil, 2) + model_NR_0)